# AP border fitting (pyABC)

In [ ]:
# required libraries
import pandas as pd
import numpy as np
from pyabc import (Distribution, RV, ABCSMC, DiscreteRandomWalkTransition, History)

In [ ]:
# Experimental data
outgrowth_df = pd.read_csv('./'+'outgrowth.csv')
outgrowth_df.set_index(['day', 'tail'], inplace=True)
outgrowth_mean = outgrowth_df.groupby('day').mean()['outgrowth']

percentage_df = pd.read_csv('./'+'percentage_100um.csv')
df = percentage_df
for day in range(0,6):
    df.loc[df['day'] == day, 'position'] = (outgrowth_mean[day] - (df.loc[df['day'] == day, 'position']-100)).astype(int)
percentage_df = df
percentage_df.set_index(['day', 'tail', 'position'], inplace=True)
percentage_df = percentage_df.drop(['unlabelled'], axis=1)
experiments = percentage_df
experiments.drop(['double','mitosis'], axis=1)

In [ ]:
# Measured AP positions
pos_list = np.arange(-1700,1000,100)

In [ ]:
# Two zones model definition
def model(pars):
    
    c1g = int(pars["c1g"])
    c2g = int(pars["c2g"])
    c1m = int(pars["c1m"])
    c2m = int(pars["c2m"])
    sp = int(pars["sp"])
    
    intensity_g_array = []
    intensity_m_array = []
    for pos in pos_list:     
        if pos <= sp:
            intensity_g = c1g
            intensity_m = c1m
        else:
            intensity_g = c2g
            intensity_m = c2m
        intensity_g_array.append(intensity_g)
        intensity_m_array.append(intensity_m)
    return {'intensity':[intensity_g_array,intensity_m_array]}

In [ ]:
# Parameters priors
c1gmin, c1gmax = 0, 100
c2gmin, c2gmax = 0, 100
c1mmin, c1mmax = 0, 100
c2mmin, c2mmax = 0, 100
spmin, spmax = -1800, 1000

In [ ]:
parameter_priors = Distribution(c1g=RV("randint", c1gmin, c1gmax-c1gmin),
                                c2g=RV("randint", c2gmin, c2gmax-c2gmin),
                                c1m=RV("randint", c1mmin, c1mmax-c1mmin),
                                c2m=RV("randint", c2mmin, c2mmax-c2mmin),
                                 sp=RV("randint", spmin, spmax-spmin))

In [ ]:
# Measured data to be compared with simulations
def measurement_data(day,tail):
    green_mean = np.asarray(percentage_df.xs(day,level='day').xs(tail,level='tail')['green'])
    magenta_mean = np.asarray(percentage_df.xs(day,level='day').xs(tail,level='tail')['magenta'])
    return [green_mean,magenta_mean]

In [ ]:
# Distances funcition definition
def distance(simulation, data):
    distance_value = np. array([])
    for sim_g,sim_m,exp_g,exp_m in zip(simulation["intensity"][0],simulation["intensity"][1],data["intensity"][0],data["intensity"][1]):
        distance_value = np. append(distance_value,np.sqrt((exp_g-sim_g)**2) + np.sqrt((exp_m-sim_m)**2))
    return np.nansum(distance_value)

In [ ]:
# Run parameters
abc = ABCSMC(models=model,
             parameter_priors=parameter_priors,
             distance_function=distance,
             population_size=1000,
             transitions=DiscreteRandomWalkTransition())

In [ ]:
# Fitting for each animal tail and day
maxiter = 30
for day,df_day in percentage_df.groupby(level='day'):
    for tail,df_animal in df_day.groupby(level='tail'):
        db_path = ("sqlite:///" + os.path.join("./fitting_results/",
                                               "sp_fitting-day="+str(day)+"-tail="+str(tail)+"_ap.db"))
        abc.new(db_path, {"intensity": measurement_data(day,tail)})
        h = abc.run(minimum_epsilon=0.1, max_nr_populations=maxiter)